In [1]:
#import matplotlib.pyplot as plt
from py4j.java_gateway import JavaGateway
import numpy as np
import struct
#%matplotlib inline

In [2]:
#from keras import callbacks
#from keras import models
#from keras import layers
#import keras.backend as K
#from keras.utils.np_utils import to_categorical

In [3]:
import tensorflow as tf

In [4]:
tf.test.is_gpu_available()

True

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 340490737383304035
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6706175017
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2139491999318611950
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [7]:
with tf.device('/CPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    # Creates a session with log_device_placement set to True.
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    # Runs the op.
    print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [1035]:
#from sklearn.model_selection import train_test_split

In [1036]:
def get_legal_moves(state, piece=None):
    if piece:
        return [list(move) for move in list(state.legalMoves(piece))]
    else:
        return [list(move) for move in list(state.legalMoves())]

In [1037]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

In [1045]:
class PolicyGradient:
    
    def __init__(self, input_dim, output_dim, learning_rate=0.01, gamma=0.99, decay_rate=0.99):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lr = learning_rate
        self.gamma = gamma
        self.decay_rate = decay_rate
        self.states = []
        self.actions = []
        self.rewards = []
        self.build_model()
        
    def pg_loss(self, y_true, y_pred):
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=model.outputs[0], labels=np.array(self.actions))
        return tf.reduce_mean(neg_log_prob * self.discounted_rewards())
        
    def build_model(self):
        model = models.Sequential()
        model.add(layers.Dense(64, activation='relu', input_shape=(self.input_dim,)))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(self.output_dim, activation=None))
        model.compile(optimizer='rmsprop', loss=self.pg_loss, metrics=['accuracy'])
        #model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
        self.model = model
        
    def log_sar(self, s, a, r):
        self.states.append(s)
        actions = np.zeros(self.output_dim)
        actions[a] = 1
        self.actions.append(actions)
        self.rewards.append(r)
        
    def choose_action(self, state, legal_moves):
        n_moves = len(legal_moves)
        probs = K.eval(K.sigmoid(self.model.predict(state[None, :])))
        return np.random.choice(n_moves, p=(probs.flatten()[:n_moves] / sum(probs.flatten()[:n_moves])))
        #return np.random.choice(len(probs.flatten()), p=probs.flatten())
    
    def discounted_rewards(self):
        discounted_rewards = np.zeros(len(self.rewards))
        total_rewards = 0
        for t in reversed(range(len(self.rewards))):
            total_rewards = total_rewards * self.gamma + self.rewards[t]
            discounted_rewards[t] = total_rewards
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)
        return discounted_rewards
    
    def train(self):
        #split = int(np.array(self.states).shape[0]*0.2)
        #x_train = np.array(self.states)[split:]
        #y_train = np.array(self.actions)[split:]
        #x_val = np.array(self.states)[:split]
        #y_val = np.array(self.actions)[:split]
        #model.fit(x_train, y_train, epochs=50, batch_size=512, validation_data=(x_val, y_val), callbacks=[early_stopping], verbose=False)
        model.fit(np.array(self.states), np.array(self.actions), epochs=50, batch_size=512, verbose=False)
        self.states = []
        self.actions = []
        self.rewards = []

In [1046]:
# class PolicyGradient:
    
#     def __init__(self, input_dim, output_dim, learning_rate=0.01, gamma=0.99, decay_rate=0.99):
#         self.input_dim = input_dim
#         self.output_dim = output_dim
#         self.lr = learning_rate
#         self.gamma = gamma
#         self.decay_rate = decay_rate
#         self.states = []
#         self.actions = []
#         self.rewards = []
#         self.build_model()
#         self.rms_prop_cache = {self.model.trainable_weights[i]: np.zeros(self.model.get_weights()[i].shape) for i in range(len(self.model.trainable_weights))}
        
        
#     def pg_loss(self, y_true, y_pred):
#         log_loss = K.categorical_crossentropy(y_true, y_pred)
#         return tf.reduce_mean(log_loss * self.discounted_rewards())
        
#     def build_model(self):
#         model = models.Sequential()
#         model.add(layers.Dense(64, activation='relu', input_shape=(self.input_dim,)))
#         model.add(layers.Dense(64, activation='relu'))
#         model.add(layers.Dense(self.output_dim, activation='softmax'))
#         #model.compile(optimizer='rmsprop', loss=self.pg_loss, metrics=['accuracy'])
#         model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
#         self.model = model
        
#     def log_sar(self, s, a, r):
#         self.states.append(s)
#         actions = np.zeros(self.output_dim)
#         actions[a] = 1
#         self.actions.append(actions)
#         self.rewards.append(r)
        
#     def choose_action(self, state, legal_moves):
#         n_moves = len(legal_moves)
#         probs = self.model.predict(state[None, :])
#         return np.random.choice(n_moves, p=(probs.flatten()[:n_moves] / sum(probs.flatten()[:n_moves])))
#         #return np.random.choice(len(probs.flatten()), p=probs.flatten())
    
#     def discounted_rewards(self):
#         discounted_rewards = np.zeros(len(self.rewards))
#         total_rewards = 0
#         for t in reversed(range(len(self.rewards))):
#             total_rewards = total_rewards * self.gamma + self.rewards[t]
#             discounted_rewards[t] = total_rewards
#         discounted_rewards -= np.mean(discounted_rewards)
#         discounted_rewards /= np.std(discounted_rewards)
#         return discounted_rewards
    
#     def train(self):
#         split = int(np.array(self.states).shape[0]*0.2)
#         x_train = np.array(self.states)[split:]
#         y_train = np.array(self.actions)[split:]
#         x_val = np.array(self.states)[:split]
#         y_val = np.array(self.actions)[:split]
#         model.fit(x_train, y_train, epochs=50, batch_size=512, validation_data=(x_val, y_val), callbacks=[early_stopping])
#         #self.states = []
#         #self.actions = []
#         #self.rewards = []
        
#     def get_gradients(self):
#         gradients = self.model.optimizer.get_gradients(self.model.total_loss, self.model.trainable_weights)
#         input_tensors = [self.model.inputs[0], # input data
#                          self.model.sample_weights[0], # how much to weight each sample by
#                          self.model.targets[0], # labels
#                          K.learning_phase(), # train or test mode
#         ]
#         get_gradients = K.function(inputs=input_tensors, outputs=gradients)
#         discounted_actions = (np.array(self.actions) * self.discounted_rewards()[:, None])
#         gradients_inputs = [np.array(self.states), [1], discounted_actions, 0]
#         return list(zip(self.model.trainable_weights, get_gradients(gradients_inputs)))
    
#     def update_weights(self):
#         grads = self.get_gradients()
#         for i, (key, val) in enumerate(grads):
#             self.rms_prop_cache[key] = self.decay_rate * self.rms_prop_cache[key] + (1 - self.decay_rate) * val**2
#             self.model.get_weights()[i] += self.lr * val / (np.sqrt(self.rms_prop_cache[key]) + 1e-5)
#         self.states = []
#         self.actions = []
#         self.rewards = []

In [1047]:
piece_to_binary = {0: [0, 0, 0],
                   1: [0, 0, 1],
                   2: [0, 1, 0],
                   3: [0, 1, 1],
                   4: [1, 0, 0], 
                   5: [1, 0, 1], 
                   6: [1, 1, 0]}
binary_to_piece = {tuple(val): key for key, val in move_to_binary.items()}

In [1048]:
gateway = JavaGateway()
tetris = gateway.entry_point.getState()

In [1049]:
input_dim = tetris.getRows() * tetris.getCols() + 3
output_dim = 34
n_iter = 50

In [1050]:
pg = PolicyGradient(input_dim, output_dim)

In [1052]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0

for i in range(n_iter):
    tetris.resetState()
    while not tetris.hasLost():
        field = np.frombuffer(tetris.getByteArray(), dtype=np.int32)
        state = np.concatenate([np.where(field>0, 1, 0), piece_to_binary[tetris.getNextPiece()]])
        moves = get_legal_moves(tetris)
        action = pg.choose_action(state, moves)
        reward = tetris.makeMove(moves[action][0], moves[action][1])
        pg.log_sar(state, action, reward)
        
episode_rewards_sum = np.sum(pg.rewards)
allRewards.append(episode_rewards_sum)
total_rewards = np.sum(allRewards)
mean_reward = np.divide(total_rewards, i+1)
maximumRewardRecorded = np.amax(allRewards)

print("==========================================")
print("Episode: ", i)
print("Reward: ", episode_rewards_sum)
print("Mean Reward", mean_reward)
print("Max reward so far: ", maximumRewardRecorded)

pg.train() 

Episode:  49
Reward:  -5000000
Mean Reward -100000.0
Max reward so far:  -5000000


In [999]:
tetris.resetState()
field = np.frombuffer(tetris.getByteArray(), dtype=np.int32)
state = np.concatenate([np.where(field>0, 1, 0), piece_to_binary[tetris.getNextPiece()]])
moves = get_legal_moves(tetris)
action = pg.choose_action(state, moves)

In [1000]:
tf.device

13

In [984]:
probs = pg.model.predict(state[None, :])

In [985]:
probs

array([[-0.05230013, -0.08159578, -0.04024404, -0.00033754,  0.00517057,
        -0.00026928, -0.05369234,  0.01551773, -0.084693  ,  0.05421113,
         0.10457634,  0.05827999,  0.11438889,  0.01350589, -0.1000725 ,
        -0.09395342, -0.02666053,  0.01096979,  0.07193859, -0.04679956,
        -0.09093128,  0.01282092, -0.00352252, -0.08819553,  0.05701552,
         0.00843234, -0.00234996,  0.02852175, -0.01426145,  0.00491546,
        -0.01046143, -0.06880352, -0.01703223, -0.00746334]],
      dtype=float32)

In [997]:
K.eval(K.sigmoid(probs))

array([[0.48692796, 0.47961235, 0.48994038, 0.4999156 , 0.50129265,
        0.49993265, 0.4865801 , 0.50387937, 0.47883943, 0.5135495 ,
        0.5261203 , 0.5145659 , 0.52856606, 0.5033764 , 0.47500274,
        0.4765289 , 0.49333528, 0.5027424 , 0.5179769 , 0.48830226,
        0.4772828 , 0.5032052 , 0.4991194 , 0.47796538, 0.51425004,
        0.50210804, 0.4994125 , 0.50712997, 0.49643466, 0.50122887,
        0.49738464, 0.4828059 , 0.49574205, 0.4981342 ]], dtype=float32)

In [940]:
pg.discounted_rewards()

array([-6.57091978e-01, -6.94032891e-01, -7.31346944e-01, -7.69037907e-01,
       -8.07109586e-01, -8.45565828e-01, -8.84410517e-01, -9.23647576e-01,
       -9.63280970e-01, -1.00331470e+00, -1.04375281e+00, -1.08459939e+00,
       -1.12585856e+00, -1.16753449e+00, -1.20963138e+00, -1.25215350e+00,
       -1.29510514e+00, -1.33849063e+00, -1.38231436e+00, -1.42658075e+00,
       -1.47129427e+00,  2.58157026e-01,  2.30461053e-01,  2.02485323e-01,
        1.74227009e-01,  1.45683258e-01,  1.16851186e-01,  8.77278816e-02,
        5.83104020e-02,  2.85957762e-02, -1.41899744e-03, -3.17369506e-02,
       -6.23611457e-02, -9.32946761e-02, -1.24540666e-01, -1.56102273e-01,
       -1.87982683e-01, -2.20185118e-01, -2.52712830e-01,  1.48904737e+00,
        1.47378464e+00,  1.45836773e+00,  1.44279510e+00,  1.42706516e+00,
        1.41117634e+00,  1.39512703e+00,  1.37891560e+00,  1.36254042e+00,
        1.34599984e+00,  1.32929218e+00,  1.31241575e+00,  1.29536886e+00,
        1.27814977e+00,  

In [941]:
np.array(pg.actions)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [942]:
(np.array(pg.actions) * pg.discounted_rewards()[:, None])

array([[-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       [-0.        , -0.        , -0.        , ..., -0.        ,
        -0.        , -0.        ],
       ...,
       [ 0.        ,  0.        ,  1.27814977, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [914]:
gradients = pg.model.optimizer.get_gradients(pg.model.total_loss, pg.model.trainable_weights)

In [915]:
input_tensors = [pg.model.inputs[0], # input data
                 pg.model.sample_weights[0], # how much to weight each sample by
                 pg.model.targets[0], # labels
                 K.learning_phase(), # train or test mode
]

get_gradients = K.function(inputs=input_tensors, outputs=gradients)

In [861]:
inputs = [np.array(pg.states), [1], np.array(pg.actions), 0]

In [862]:
(get_gradients(inputs)[0])

array([[-5.9459945e-03, -2.0868776e-03,  4.6095811e-04, ...,
         1.2176041e-05,  1.8728647e-04, -5.9732440e-04],
       [-8.9348974e-03, -2.9007015e-03,  3.7169934e-04, ...,
         4.2192737e-04,  9.7620138e-04, -6.3371408e-04],
       [-9.3236081e-03, -3.6692703e-03,  6.2631327e-04, ...,
         1.3859002e-03,  9.2979847e-04, -1.4759435e-03],
       ...,
       [-8.6866645e-03, -3.1647703e-03,  3.4829124e-04, ...,
        -7.0223899e-04,  1.6135559e-03, -2.6759827e-03],
       [-3.2586919e-03, -1.7304625e-03,  5.7548285e-04, ...,
        -3.0119423e-04,  1.4157308e-03, -3.1432020e-04],
       [-8.9635234e-03, -2.5400408e-03,  4.5889570e-05, ...,
        -3.3217022e-04,  7.7794970e-04, -2.1964838e-03]], dtype=float32)

In [837]:
gamma = 0.9

In [863]:
pg.model.layers

In [864]:
{pg.model.trainable_weights[i]: np.zeros(pg.model.get_weights()[i].shape) for i in range(len(pg.model.trainable_weights))}

{<tf.Variable 'dense_44/kernel:0' shape=(213, 64) dtype=float32_ref>: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 <tf.Variable 'dense_44/bias:0' shape=(64,) dtype=float32_ref>: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 <tf.Variable 'dense_45/kernel:0' shape=(64, 64) dtype=float32_ref>: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 <tf.Variabl

In [807]:
rms_prop_cache = {}
for i in range(6):
    rms_prop_cache[pg.model.trainable_weights[i]] = np.zeros(pg.model.get_weights()[i].shape)

In [875]:
grads = pg.get_gradients()

In [876]:
for i, (key, val) in enumerate(grads):
    pg.rms_prop_cache[key] = pg.decay_rate * pg.rms_prop_cache[key] + (1 - pg.decay_rate) * val**2
    pg.model.get_weights()[i] += pg.lr * val / (np.sqrt(pg.rms_prop_cache[key]) + 1e-5)

array([[3.53548489e-06, 4.35505825e-07, 2.12482387e-08, ...,
        1.48255973e-11, 3.50762197e-09, 3.56796441e-08],
       [7.98323890e-06, 8.41406916e-07, 1.38160408e-08, ...,
        1.78022699e-08, 9.52969117e-08, 4.01593532e-08],
       [8.69296764e-06, 1.34635445e-06, 3.92268298e-08, ...,
        1.92071937e-07, 8.64525234e-08, 2.17840920e-07],
       ...,
       [7.54581424e-06, 1.00157717e-06, 1.21306787e-08, ...,
        4.93139645e-08, 2.60356273e-07, 7.16088323e-07],
       [1.06190726e-06, 2.99450079e-07, 3.31180523e-08, ...,
        9.07179665e-09, 2.00429369e-07, 9.87971926e-09],
       [8.03447529e-06, 6.45180705e-07, 2.10585258e-10, ...,
        1.10337055e-08, 6.05205770e-08, 4.82454141e-07]])

In [843]:
pg.lr * val / (np.sqrt(rms_prop_cache[key]) + 1e-5)

array([[-5.94599442, -2.08687761,  0.46095811, ...,  0.01217604,
         0.18728646, -0.59732438],
       [-8.93489705, -2.90070147,  0.37169932, ...,  0.42192737,
         0.97620132, -0.63371408],
       [-9.32360781, -3.66927015,  0.62631325, ...,  1.38590012,
         0.92979844, -1.47594346],
       ...,
       [-8.68666466, -3.16477017,  0.34829125, ..., -0.70223896,
         1.61355583, -2.67598261],
       [-3.25869187, -1.73046246,  0.57548282, ..., -0.30119422,
         1.41573073, -0.31432019],
       [-8.96352358, -2.54004062,  0.04588957, ..., -0.33217023,
         0.77794966, -2.19648373]])

In [848]:
pg.model.get_weights()[0]

array([[ 0.08740462,  0.05523278, -0.14190304, ..., -0.07093447,
        -0.13969295, -0.0912168 ],
       [-0.02309598, -0.08283542, -0.12689406, ..., -0.08007182,
        -0.01867096,  0.06120569],
       [ 0.11393288, -0.06422535,  0.07250626, ...,  0.09786937,
        -0.04573769,  0.14054278],
       ...,
       [ 0.09221661, -0.10187833, -0.0203408 , ...,  0.02046031,
        -0.12427407,  0.05350097],
       [-0.098893  ,  0.07750121,  0.06017603, ...,  0.00875151,
         0.06151143, -0.09341399],
       [ 0.12461805, -0.08963294, -0.12624465, ...,  0.13119504,
        -0.09563239,  0.04821433]], dtype=float32)

In [806]:
pg.model.get_weights()

[array([[ 0.08740462,  0.05523278, -0.14190304, ..., -0.07093447,
         -0.13969295, -0.0912168 ],
        [-0.02309598, -0.08283542, -0.12689406, ..., -0.08007182,
         -0.01867096,  0.06120569],
        [ 0.11393288, -0.06422535,  0.07250626, ...,  0.09786937,
         -0.04573769,  0.14054278],
        ...,
        [ 0.09221661, -0.10187833, -0.0203408 , ...,  0.02046031,
         -0.12427407,  0.05350097],
        [-0.098893  ,  0.07750121,  0.06017603, ...,  0.00875151,
          0.06151143, -0.09341399],
        [ 0.12461805, -0.08963294, -0.12624465, ...,  0.13119504,
         -0.09563239,  0.04821433]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[ 0.16237243, -0.19294907,  0.14524777, ..., 

In [712]:
tetris.resetState()

True

In [713]:
piece_to_binary[tetris.getNextPiece()]

[0, 0, 1]

In [714]:
moves = get_legal_moves(tetris)

In [715]:
moves

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [1, 0],
 [1, 1],
 [1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6]]

In [670]:
tetris.makeMove(1, 7)

0

In [382]:
traj = trajectories[0]

In [385]:
t = 0

In [386]:
traj[t]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), [0, 2])

In [398]:
state.gebinary_to_move[tuple(traj[t][0][-3:])]

4

In [414]:
moves = get_legal_moves(state, binary_to_piece[tuple(traj[t][0][-3:])])

In [416]:
j = 0

In [421]:
inputs = traj[t][0]

In [429]:
inputs[0] - sum(boltzmann_distribution(inputs, theta) * inputs[0])

0.0

In [277]:
theta = np.random.uniform(0, 0.1, (len(inputs), max(move_lens.values())))

1

In [268]:
[list(move) for move in list(state.legalMoves())]

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [1, 0],
 [1, 1],
 [1, 2],
 [1, 3],
 [1, 4],
 [1, 5],
 [1, 6],
 [1, 7],
 [2, 0],
 [2, 1],
 [2, 2],
 [2, 3],
 [2, 4],
 [2, 5],
 [2, 6],
 [2, 7],
 [2, 8],
 [3, 0],
 [3, 1],
 [3, 2],
 [3, 3],
 [3, 4],
 [3, 5],
 [3, 6],
 [3, 7]]

In [80]:
move[0] = 3
move[1] = 1

In [85]:
state.getRows()

21

In [86]:
state.getCols()

10

In [29]:
int.from_bytes(state.getByteArray(), byteorder='big')

0

In [23]:
state.getRows()

21

In [24]:
state.getCols()

10

In [20]:
np.reshape(list(state.getByteArray()), (state.getRows(), state.getCols()))

ValueError: cannot reshape array of size 840 into shape (21,10)

In [58]:
state.getRows()

21

In [17]:
state.getCols()

10

In [17]:
state.makeMove(state.legalMoves()[0])

In [18]:
state.draw()

Py4JJavaError: An error occurred while calling o1.draw.
: java.lang.NullPointerException
	at State.draw(State.java:262)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.base/java.lang.Thread.run(Thread.java:834)


In [53]:
(state.getpBottom())

JavaObject id=o296

In [29]:
list(list([list(item) for item in list(state.getpBottom())][0])[0])

[0, 0]

In [52]:
state.ROWS

In [43]:
gateway.get_field(state, ROWS)

NameError: name 'ROWS' is not defined

In [46]:
gateway.

AttributeError: 'JavaMember' object has no attribute '_get_object_id'